In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$



Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавая свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками.

Класс MeanTargetEncoderNoise должен иметь следующую сигнатуру:



In [ ]:
temp[categorical] = temp[categorical].map(self.dict_of_means[categorical] + C * self.noise[categorical]).fillna(0)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):              
        ### Your code is here

    def fit(self, X, y):
        ### Your code is here
        
        
        return self
        


    def transform(self, df):
        ### Your code is here
        
        return temp

Разделите колонки на вещественные и категориальные. Приведите все категориальные колонки к типу `object`.

Далее применим наш кодировщик к `X_train, X_test`, так же как например мы применяем `StandardScaler`, чтобы проверить работоспособность нашего класса. Установите зерно датчика случайный чисел `np.random.seed(1)`.

После того, как вы изменили обучающую и тестовую выборки, сохраните первые 10 строк полученного промежуточного датафрейма обучающей выборки (`X_train`) в файл в формате csv с сепаратором `;`. Не забудьте индекс. Отправьте полученный файл в форму ниже.

Список колонок которые должны быть в файле для сдачи:
```py
cols = [
    "km_driven",
    "name",
    "year",
    "fuel",
    "seller_type",
    "transmission",
    "owner"
]
```

### Ваше решение


Разделение колонок на категориальные и числовые.

In [5]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']


X[object_cols] = X[object_cols].astype(object)
X_train[object_cols] = X[object_cols].astype(object)
X_test[object_cols] = X[object_cols].astype(object)

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


Реализация класса MeanTargetEncoderNoise.

In [6]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):

    def __init__(self, categorical, numeric):
        ### Your code is here
        self.categorical = categorical
        self.numeric = numeric

    def fit(self, X, y):
      
        X_fit = X.copy()
        y_fit = y.copy()

       
        X_with_target = pd.concat((X_fit, y_fit), axis=1)

        # Задаем имя таргета
        self.target_name = y_fit.name

        
        self.dict_of_means = {col : X_with_target.groupby(col)[self.target_name].mean()
                              for col in self.categorical}

        
        self.noise = {col: np.random.normal(loc=0, scale=1, size = X_with_target[col].nunique())
                      for col in self.categorical}
       
        return self

    def transform(self, df):

        temp = df.copy()
        C = 0.006

        for numeric in self.numeric:
            temp[numeric] = temp[numeric].fillna(0)

        for categorical in self.categorical:

            temp[categorical] = temp[categorical].map(self.dict_of_means[categorical] + C * self.noise[categorical]).fillna(0)

        return temp[["km_driven", "name", "year", "fuel", "seller_type", "transmission", "owner"]]

Проверка работы трансформера.

In [7]:
np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

train.head(10)

,km_driven,name,year,fuel,seller_type,transmission,owner
3294,50000,13.483692,13.436559,13.093756,12.615863,13.771135,12.977730
2290,70000,12.117029,11.903115,12.453832,12.615863,13.771135,12.977730
874,50000,12.310456,13.328864,12.453832,12.615863,12.639805,12.977730
1907,92198,12.491443,13.042359,12.453832,13.152824,12.639805,12.463313
3244,3240,12.390906,12.870886,12.453832,12.615863,12.639805,12.463313
1089,10000,12.687432,13.436559,12.453832,13.152824,12.639805,12.977730
3902,90000,11.698702,11.503504,12.453832,12.615863,12.639805,11.878390
2215,79000,11.120678,11.503504,12.453832,12.615863,12.639805,12.463313
3862,99700,13.172879,13.328864,13.093756,12.615863,12.639805,12.977730
705,124000,13.004194,12.241213,13.093756,12.615863,12.639805,12.463313


In [8]:
train.head(10).reset_index(drop = True).to_csv('task5.csv', sep=';', index= False)

Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [9]:
max_depth_list = [3, 5, 8, 12]
min_samples_split_list = [10, 50, 100, 500]
min_impurity_decrease_list = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_list = [100, 200, 500]

In [10]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here



pipes_max_depth_list = []
### Your code is here
for max_depth in max_depth_list:
    pipe = Pipeline(
        [
            ('transform', MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)),
            ('decision_tree', DecisionTreeRegressor(max_depth=max_depth))
        ]
    )

    pipes_max_depth_list.append(pipe.fit(X_train, y_train))


mse_list = []

for pipe in pipes_max_depth_list:    
    preds = pipe.predict(X_test)
    #print(mse(y_test, preds))
    mse_list.append(mse(preds, y_test))

print(round(min(mse_list),2))



0.8


Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "decision_tree__max_depth": [3, 5, 8, 12],
    "decision_tree__min_samples_split": [10, 50, 100, 500],
    "decision_tree__min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "decision_tree__max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

### Your code is here



